# A선박 전체 전처리 파이프라인 및 시각화 

In [1]:
1

1

In [ ]:
# 데이터 row 값 생략 없이 모두 출력하기
#pd.set_option('display.max_rows', None)
# 초기화
pd.reset_option('display.max_rows')

'''
중간 중간 주요 필터링 함수에서 Log가 찍히도록 추가하면 더 좋을 것 같습니다. (tqdm을 사용하면 더 좋겠죠)
나머지는 훌륭합니다.
'''

In [ ]:
import pandas as pd
import folium as f
import os
import glob
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
import datetime
import random
import numpy as np
from haversine import haversine
from sklearn.cluster import DBSCAN
import itertools
from tqdm import tqdm

In [ ]:
all_csv = glob('/STORAGE/DATA/01_KRISO_G_MAPPED_DATA/Position_LTEM/*.csv')
all_csv = sorted(all_csv)
selected_csv = all_csv[0:25]

total = pd.DataFrame()

for sel in tqdm(selected_csv):
    temp = pd.read_csv(sel, sep = ',', encoding='utf-8')
    total = pd.concat([total, temp])

In [ ]:
# key 값은 A선박 종류 이름, value는 해당 dataframe을 받았다.

start_a_ship = total[total.SHIP_CODE.str.startswith('A')].SHIP_CODE.unique()

# key 값을 어선 종류, value 값을 해당 어선의 dataframe 으로 받게끔 했다.
dict_ = {}

for a_ship in start_a_ship:
    value  = total[total.SHIP_CODE.values == a_ship]
    dict_[a_ship] = value

# ---------------------------------

In [ ]:
# SOG >= 2 Data frame 
def upper_2_dataframe(df):
    # Dict.values 안에 있는 dataframe -> list
    dict_to_lst = []
    
    for df in df.values():
        dict_to_lst.append(df[df.dSOG.values >=2])
    
    # SOG <2 제외 하고 데이터 프레임 갱신
    not_empty_df = [full for full in dict_to_lst if not full.empty]
    
    return not_empty_df

In [ ]:
# 거리 파생변수 추가해주는 dataframe
def add_distance_df(df):
    
    for i in range(len(df)):
        dLat = df[i].dLat.tolist()
        dLon = df[i].dLon.tolist()
        position = []              # haversine 을 사용하기 위한 위, 경도 결합 리스트
        dist = []  # haversine 을 사용하여 직선거리를 담아 둘 리스트 (파생변수 담아 둘 리스트)
        
        for pos in zip(dLat, dLon):
            position.append(pos)

        for j in range(1, len(position)):
            dist.append(haversine(position[j-1], position[j]) * 1000) # haversine 은 default 가 km 이다.

        df[i]['dist'] = np.mean(dist)
        df[i]['dist'][1:] = dist
        
        position.clear()
        dist.clear()
    
    return df

In [ ]:
# 이전 좌표와의 거리 10 이상 df
def dist_upper_10_df(df):
    for i in range(len(df)):
        df[i] = df[i][df[i].dist.values > 10]
    return df

In [ ]:
# 시간을 보기좋게 전처리 (tooltip)
def reset_index_time_series(df):
    for i in range(len(df)):
        time_int = df[i].szMsgSendDT.values.tolist()
        time_str = []
        timestamp_ = []

        for j in range(len(time_int)):
            time_str.append(str(time_int[j]))
        # 시간 정보를 보기 좋게 바꿔준다
        for str_ in time_str:
            n = datetime.datetime.strptime(str_,"%Y%m%d%H%M%S%f")
            timestamp_.append(n.strftime("%Y-%m-%d-%H-%M-%S"))
        df[i]['szMsgSendDT'] = timestamp_
        
        time_str.clear()
        timestamp_.clear()

    return df

In [ ]:
# COG 간격 컬럼 추가
def add_cog_interval(df):
    for i in range(len(df)):
        df[i]['dCOG_diff'] = abs(df[i]['dCOG'].diff())
    return df

In [ ]:
'''
B선박
'''
upper_2 = upper_2_dataframe(dict_)
reset_time = reset_index_time_series(upper_2)

# 최종 전처리 셋 : 모든 A 선박, 하나의 DATAFRAME 으로 저장 
final_preprocessing  = pd.DataFrame()

for each_B_ship_df in reset_time:
    final_preprocessing = pd.concat([final_preprocessing, each_B_ship_df])
    
final_preprocessing.reset_index(inplace = True, drop = True)

import plotly.express as px
import plotly.offline as pyo

fig = px.line_mapbox(final_preprocessing, lat = "dLat", lon = "dLon", color = 'SHIP_CODE', zoom =3, height=900)

# 지도 배경 layout 토큰
fig.update_layout(mapbox=dict(
    accesstoken='pk.eyJ1IjoibGFnb29uNiIsImEiOiJjbGd4M2gycmkwM3VzM3JscGQ0YzlxYjU1In0.CBy01IB-Z8klhIcFRnkXKg',
    zoom=3
))
    
pyo.plot(fig, filename = 'B선박 전체동선(전처리 전).html')

In [ ]:
# 직선에 가장 가까운 상위 3가지 동선만 남기기
def similar_straight_cog(df):
    
    flag = True
    
    for i in range(len(df)):
        df[i] = df[i].reset_index()
        df[i].drop(labels = 'index', axis = 1, inplace = True)
        
        # 이전 좌표와 COG 간격 차이가 20이상이 되는 순간들의 index 좌표들만 추출
        index_lst = []
        for idx, col in df[i].iterrows():
            if col['dCOG_diff'] >= 20:
                index_lst.append(idx)
        
        # 직선에 가까운 상위 3가지 동선 (인덱스 차이) 추출
        interval_lst = []
        for j in range(1,len(index_lst)):
            interval_lst.append(index_lst[j] - index_lst[j-1])
        sorted_interval_lst = sorted(interval_lst, reverse = True)
        sort_3 = sorted_interval_lst[:3]


        final = []
        for k in range(1, len(index_lst)):
            if index_lst[k] - index_lst[k-1] in sort_3:
                final.append([index_lst[k-1], index_lst[k]])

        indexes_to_keep = []
        for start, end in final:
            indexes_to_keep.extend(list(range(start, end + 1)))
        
        # 두번째 데이터 프레임 부터는 리스트를 초기화 하지 않는다.
        if flag :
            filtered_df = []

        # 인덱스에 해당 행만 남기고 나머지 행 제거
        filtered_df.append(df[i].iloc[indexes_to_keep])
        flag = False
    
    return filtered_df

In [ ]:
upper_2 = upper_2_dataframe(dict_)
add_dist = add_distance_df(upper_2)
dist_upper_10 = dist_upper_10_df(add_dist)
reset_time = reset_index_time_series(dist_upper_10)
add_cog = add_cog_interval(reset_time)
straigt_line = similar_straight_cog(add_cog)

In [ ]:
# 최종 전처리 셋 : 모든 A 선박, 하나의 DATAFRAME 으로 저장 
final_preprocessing  = pd.DataFrame()

for each_A_ship_df in straigt_line:
    final_preprocessing = pd.concat([final_preprocessing, each_A_ship_df])
    
final_preprocessing

# 대용량 시각화 Datashaher , Plotly

In [ ]:
import plotly.express as px
import plotly.offline as pyo

fig = px.line_mapbox(final_preprocessing, lat = "dLat", lon = "dLon", color = 'SHIP_CODE', zoom =3, height=900)

# 지도 배경 layout 토큰
fig.update_layout(mapbox=dict(
    accesstoken='pk.eyJ1IjoibGFnb29uNiIsImEiOiJjbGd4M2gycmkwM3VzM3JscGQ0YzlxYjU1In0.CBy01IB-Z8klhIcFRnkXKg',
    zoom=3
))
    
pyo.plot(fig, filename = 'plotly_LINE_전처리.html')

In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

all_lats = []
all_lons = []
for i in range(len(straigt_line)):
    lats = straigt_line[i].dLat
    lons = straigt_line[i].dLon
    
    
    all_lats.extend(lats)
    all_lons.extend(lons)
    
# Create a scatter mapbox trace
trace = go.Scattermapbox(
    lat=all_lats,
    lon=all_lons,
    mode='markers',
    marker=dict(
        size=9,
        color='#ECD5E3',
        opacity=0.7
    ),
)

# Create a layout
layout = go.Layout(
    mapbox=dict(
        accesstoken='pk.eyJ1IjoibGFnb29uNiIsImEiOiJjbGd4M2gycmkwM3VzM3JscGQ0YzlxYjU1In0.CBy01IB-Z8klhIcFRnkXKg',
        ),
        zoom=3
    )
)

# Create a figure
fig = go.Figure(data=[trace], layout=layout)

# Create html file
pyo.plot(fig, filename = 'plotly_Marker_전처리.html')

In [ ]:
import holoviews as hv, colorcet as cc
import datashader as ds
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

In [ ]:
'''
x,y = ds.utils.lnglat_to_meters(final_preprocessing.dLon, final_preprocessing.dLat)
final_preprocessing['dLon'] = x
final_preprocessing['dLat'] = y

해당 코드는 경도 위도 -> 윕머케이터 좌표계 변환 코드임
따라서, 데이터 프레임 변동이 없다면 처음에만 실행, 이후 실행 시 주석 처리 필수
'''

In [ ]:
# 최종 전처리 파일 Datashader visualization
x,y = ds.utils.lnglat_to_meters(final_preprocessing.dLon, final_preprocessing.dLat)
final_preprocessing['dLon'] = x
final_preprocessing['dLat'] = y

map_tiles = hv.element.tiles.EsriImagery().opts(alpha=0.5, width=600, height=600)

points = hv.Points(final_preprocessing, ['dLon', 'dLat'])

ship_traffic = datashade(points, cmap= 'magma', width=600, height=600, x_sampling=1, y_sampling=1)
datashade_visualization = map_tiles * ship_traffic
hv.save(datashade_visualization, 'final_preprocessing_datashade_.html')

In [ ]:
# 전처리 이전 1600만 row 파일 Datashader visualization
x_,y_ = ds.utils.lnglat_to_meters(total.dLon, total.dLat)
total['dLon'] = x_
total['dLat'] = y_

map_tiles_ = hv.element.tiles.EsriImagery().opts(alpha=0.5, width=600, height=600)

points_ = hv.Points(total, ['dLon', 'dLat'])

ship_traffic_ = datashade(points_, cmap= 'magma', width=600, height=600, x_sampling=1, y_sampling=1)
datashade_visualization_ = map_tiles_ * ship_traffic_
hv.save(datashade_visualization_, 'total_datashade.html')

# 전처리 걸린 시간

In [ ]:
import timeit
start = timeit.default_timer()
upper_2 = upper_2_dataframe(dict_)
a1 = timeit.default_timer()
add_dist = add_distance_df(upper_2)
a2 = timeit.default_timer()
dist_upper_10 = dist_upper_10_df(add_dist)
a3 = timeit.default_timer()
reset_time = reset_index_time_series(dist_upper_10)
a4 = timeit.default_timer()
add_cog = add_cog_interval(reset_time)
a5 = timeit.default_timer()
straigt_line = similar_straight_cog(add_cog)
end = timeit.default_timer()

print(f'전처리 전과정 수행 시간 : {end-start} sec')

print('--------------------------')
print(f'upper_2_dataframe 함수 실행 시간 : {a1 - start} sec')
print(f'add_distance_df 함수 실행 시간 : {a2 - a1} sec')
print(f'dist_upper_10_df 함수 실행 시간 : {a3 - a2} sec')
print(f'reset_time_series 함수 실행 시간 : {a4 - a3} sec')
print(f'add_cog_interval 함수 실행 시간 : {a5 - a4} sec')
print(f'similar_straight_cog 함수 실행 시간 : {end - a5} sec')

In [ ]:
straigt_line[0][straigt_line[0].duplicated()]

# ---------------------------------------------------------------------------

# COG 기준 값 설정 (EDA)

In [ ]:
# cog_interval의 전처리 기준 값을 정하기 위해 cog_interval 의 분포를 알아보기 위한 코드
plt.figure(figsize = (10,6))

plt.subplot(3,1,1)
sns.countplot(x = 'dCOG_diff', data = cog_interval)
plt.xlim([0,50])
plt.xticks(rotation = 90)
plt.plot()
plt.title('cog_interval frequency')
plt.legend(cog_interval.SHIP_CODE)


plt.subplot(3,1,2)
sns.countplot(x = 'dCOG_diff', data = cog_interval)
plt.xlim([10,50])
plt.xticks(np.arange(10,50),rotation = 90)
plt.plot()
plt.title('cog_interval frequency (range : 10~50)')

plt.subplot(3,1,3)
sns.countplot(x = 'dCOG_diff', data = cog_interval)
plt.xlim([0,10])
plt.xticks(np.arange(0,10),rotation = 90)
plt.plot()
plt.title('cog_interval frequency (range : 0~10)')

plt.subplots_adjust(hspace = 1.2)

plt.show()